# Example notebook for the Rigol DG 1062 instrument

In [1]:
from qcodes.instrument_drivers.rigol.DG1062 import DG1062

In [2]:
gd = DG1062("gd", "TCPIP0::169.254.187.100::INSTR")

Connected to: Rigol Technologies DG1062Z (serial:DG1ZA195006397, firmware:03.01.12) in 0.13s


In [3]:
gd.channels[0].apply()

{'waveform': 'SIN', 'freq': 2000.0, 'ampl': 1.0, 'offset': 0.0, 'phase': 0.0}

In [4]:
gd.channels[0].freq()

2000.0

In [5]:
gd.channels[0].waveform()  # We can only querry waveform 

'SIN'

In [6]:
gd.channels[0].sin(freq=2E3, ampl=1.0, offset=0, phase=0)

{}

In [7]:
help(gd.channels[0].sin)

Help on function f in module qcodes.instrument_drivers.rigol.DG1062:

f(**kwargs)
    Arguments: ['freq', 'ampl', 'offset', 'phase']



In [8]:
help(gd.channels[0].dc)

Help on function f in module qcodes.instrument_drivers.rigol.DG1062:

f(**kwargs)
    Arguments: ['freq', 'ampl', 'offset']



In [9]:
help(gd.channels[0].arb)

Help on function f in module qcodes.instrument_drivers.rigol.DG1062:

f(**kwargs)
    Arguments: ['sample_rate', 'ampl', 'offset']



In [9]:
try: 
    gd.channels[0].waveform("SIN")
except: 
    print("We cannot set a waveform like this!")

We cannot set a waveform like this!


In [10]:
gd.channels[0].dc(freq=0, ampl=0, offset=1)

{}

Put a square wave on channel 2

In [12]:
gd.channels[1].squ(freq=1E3, ampl=1.0, offset=0, phase=0)

{}

Querry the freqency on channel 1

In [7]:
gd.channels[0].freq()

'DEF'

In [8]:
gd.channels[1].freq()

1000.0

Now lets try to set the sample rate. Since the sample rate is only a parameter for the "ARB" waveform, we will get a warning 

In [7]:
gd.channels[0].sample_rate(1)

Warning, unable to set 'sample_rate' for the current waveform


Try to If we try to apply a waveform but do not give sufficient arguments, an exception is raised. 

In [11]:
gd.channels[0].sin(freq=1.5E3, ampl=1.0, offset=0)

ValueError: Waveform SIN needs at least parameters ['freq', 'ampl', 'offset', 'phase']

Likewise an exception is raised if we use an unknown waveform 